In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pickle
import gzip
import time
from warnings import filterwarnings
filterwarnings('ignore')
from _utils import utils, transforming, filling

data = ['../BigContest_data/' + file for file in os.listdir('../BigContest_data')]

In [2]:
# 보간으로 채워넣은 데이터
concat_log = pd.read_csv('../preprocessed/concat_log.csv')
user_spec = pd.read_csv('../preprocessed/user_spec_filled.csv')
loan_result = pd.read_csv(data[2])

In [3]:
# 가설검정 및 EDA는 test data(6월)를 제외한 train data만으로 하기로 한다.
user_spec_train, _ = utils.split_train_test(user_spec, 'insert_time')
loan_result_train, _ = utils.split_train_test(loan_result, 'loanapply_insert_time')

# 가설 확인
## 가설 1.
- 중금리 대출신청하는 핀다 주요고객은 신용이 부족한 사람, 이미 시중은행에서 신용대출을 이미 최대로 받은 사람일 확률이 높을 것이다.
### 1-1
- 사회초년생, 저신용점수자, 비정기소득자 등 신용이 부족한 사람일수록 대출비교 신청이 많을 것이다.
### 1-2
- 이미 시중은행 대출을 최대한도로 사용한 사람 즉, 기대출 건수 및 기대출 금액이 큰 사람일수록 대출비교신청이 많을 것이다.

In [ ]:
print(f'신용점수 평균 : ',user_spec_train['credit_score'].mean() )
print(f'신용점수 중앙값 : ',user_spec_train['credit_score'].median() )
fig, ax = plt.subplots(1,2)
user_spec_train['credit_score'].plot(kind='box', ax=ax[0])
user_spec_train['existing_loan_amt'].plot(kind='box', ax=ax[1])

In [36]:
applied_application_id = loan_result_train.query('is_applied==1')['application_id'].unique()
user_spec_train['is_applied'] = user_spec_train['application_id'].map(lambda x:1 if x in applied_application_id else 0)

In [47]:
company_enter_period = 2022-pd.to_datetime(user_spec_train['company_enter_month'].map(utils.make_date_format)).dt.year

In [ ]:
cond1 = company_enter_period <= 1 
cond2 = (company_enter_period <= 3) | (company_enter_period > 1)
cond3 = company_enter_period > 3

print('1년차 이하')
display(user_spec_train.loc[cond1,'is_applied'].value_counts(normalize=True))
print('3년차 이하')
display(user_spec_train.loc[cond2,'is_applied'].value_counts(normalize=True))
print('그외')
display(user_spec_train.loc[cond3,'is_applied'].value_counts(normalize=True))

In [ ]:
credit_score_cut = pd.cut(user_spec_train['credit_score'], range(0,1100,100), labels=np.arange(0,1000,100)+1)
credit_score_cut = credit_score_cut.astype(object)
user_spec_train.groupby(credit_score_cut)['is_applied'].apply(lambda x:x.value_counts(normalize=True)).unstack(level=1)

In [ ]:
user_spec_train.groupby('income_type')['is_applied'].apply(lambda x:x.value_counts(normalize=True)).unstack(level=1)

## 가설 2.
- 대출 목적별로 대출희망금액이 상이할 것이다.
### 2-1
- 대출목적별 대출희망 금액 분포에 차이가 있을 것이다.
- 대출목적별로 대출희망 금액이 평균에 가까울수록 대출비교신청이 많을 것이다.

In [ ]:
mapping = {'생활비':'LIVING','대환대출':'SWITCHLOAN','기타':'ETC','투자':'INVEST',
          '사업자금':'BUSINESS','자동차구입':'BUYCAR','전월세보증금':'HOUSEDEPOSIT','주택구입':'BUYHOUSE'}
user_spec_train['purpose'] = user_spec_train['purpose'].replace(mapping)
user_spec_train.groupby('purpose')['desired_amount'].agg(['max','mean','median','min'])


In [ ]:
sns.boxplot(data=user_spec_train, x='purpose', y='desired_amount')
plt.yscale('log')
plt.xticks(rotation =90)

## 가설 3.
- 금리가 낮을수록 대출신청률이 높을 것이다.

In [ ]:
loan_result_train.head()

In [ ]:
sns.boxplot(data=loan_result_train, x = 'is_applied', y = 'loan_rate')

## 가설 4.
- 대출희망 금액을 최대한 맞출 수 있도록 대출한도를 조합하여 대출신청을 할 것이다.<br>
(즉, 대출신청한 경우 신청한 상품들의 대출한도 합계가 대출희망금액과 유사할 것이다.)

In [60]:
loan_result_train = pd.merge(loan_result_train,user_spec_train[['application_id','desired_amount']], on='application_id', how='left')

In [75]:
loan_result_train['ratio'] = loan_result_train['loan_limit'] / loan_result_train['desired_amount'].replace(0, np.nan)

In [77]:
loan_temp = loan_result_train.query('is_applied==1')
loan_desired_amount_sum = loan_temp.groupby('application_id')['loan_limit'].sum()

In [ ]:
print(f'비율의 평균 : {loan_result_train["ratio"].mean()}')
pd.concat([loan_desired_amount_sum,loan_temp.groupby('application_id')['desired_amount'].mean(),
          loan_temp.groupby('application_id')['ratio'].mean()],axis=1).head()


## 가설 5.
- 은행에 대한 선호도가 대출신청 여부에 영향을 줄 것이다.<br>
(즉, 은행별 CVR\*에 차이가 있을 것이다.)

(\*CVR : Conversion Rate = 대출신청횟수 / 대출비교노출횟수)

In [ ]:
(loan_result.query('is_applied==1').groupby('bank_id')['application_id'].count() / loan_result['bank_id'].value_counts().sort_index()).sort_values(ascending=False)

## 가설 6.
- 특정 서비스 이용률이 높을수록 대출신청률도 높을 것이다.
- \~~한 행동을 하면 대출 신청을 하는지 보기 위함
- 마지막 is_applied_Y 이후의 데이터는 지워주지만 applied가 아예 없는 경우는 남김

In [ ]:
def slice_last_is_applied(x):
    x_list = x.tolist()
    if 'is_applied_Y' in x_list:
        last_index = lambda x:len(x)-x[::-1].index('is_applied_Y')-1
        return x.iloc[:last_index(x_list)+1].tolist()
    else:
        return x_list
temp1 = concat_log.groupby('user_id')['event'].agg(slice_last_is_applied)
temp2 = temp1.explode().reset_index()
temp2['temp'] = 1
event_count = temp2.groupby(['user_id','event']).count()

pattern_search = event_count.reset_index().pivot(index='user_id',columns='event',values='temp')
pattern_binary = pattern_search.applymap(lambda x:1 if x>=1 else 0)
pattern_binary.head()

In [ ]:
# jacard : 겹치는 비율이 이 정도 된다.
# recall : 대출 승인 받은 사람 중 이 정도 비율의 사람들은 해당 행동을 했다.
# precision : 해당 행동을 한 사람들 중에 이 정도 비율로 대출 승인을 받았다.

from sklearn.metrics import jaccard_score,recall_score, precision_score
pd.concat([
        pattern_binary.apply(lambda x:jaccard_score(x,pattern_binary['is_applied_Y']),axis=0),
        pattern_binary.apply(lambda x:recall_score(pattern_binary['is_applied_Y'],x),axis=0),
        pattern_binary.apply(lambda x:precision_score(pattern_binary['is_applied_Y'],x),axis=0)
    ],axis=1).rename(columns = {0:'jaccard',1:'recall',2:'precision'})

# 주요 is_applied 시간대

In [91]:
is_applied1 = pd.to_datetime(loan_result_train.query('is_applied==1')['loanapply_insert_time'])
is_applied0 = pd.to_datetime(loan_result_train.query('is_applied==0')['loanapply_insert_time'])

In [ ]:
# 월별 대출 신청 비율
pd.concat([
    is_applied1.dt.month.value_counts(normalize=True).rename('신청O 비율'),
    is_applied0.dt.month.value_counts(normalize=True).rename('신청X 비율'),
    is_applied1.dt.month.value_counts(normalize=False).rename('신청O 건수').map(utils.split_comma),
    is_applied0.dt.month.value_counts(normalize=False).rename('신청X 건수').map(utils.split_comma)
],axis=1).sort_index()

In [ ]:
# 일별 대출 신청 비율
pd.concat([is_applied1.dt.day.value_counts(normalize=True).rename('Y'),
           is_applied0.dt.day.value_counts(normalize=True).rename('N')
          ],axis=1).sort_index().plot(kind='bar')
plt.xlabel('day')
plt.ylabel('ratio')
plt.show()

In [ ]:
# 요일별 대출 신청 비율
pd.concat([is_applied1.dt.weekday.value_counts(normalize=True).rename('Y'),
           is_applied0.dt.weekday.value_counts(normalize=True).rename('N')
          ],axis=1).sort_index().plot(kind='bar')
plt.xlabel('weekday')
plt.ylabel('ratio')
plt.xticks(range(7),labels = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.show()

In [ ]:
# 시간대별 대출 신청 비율
timestamp = pd.to_datetime(loan_result_train['loanapply_insert_time'])
timestamp_int = timestamp.dt.hour + timestamp.dt.minute/60 + timestamp.dt.second/3600
sns.violinplot(x=loan_result_train['is_applied'],y=timestamp_int.rename('timestamp'), scale='area')
plt.xticks(range(2),labels=['Y','N'])
plt.show()

In [ ]:
# 월,일,시간,분,초 모두 고려한 전 기간
timestamp_all = timestamp.dt.month + (timestamp.dt.day + (timestamp.dt.hour + timestamp.dt.minute/60 + timestamp.dt.second/3600)/24)/30
sns.violinplot(x=loan_result_train['is_applied'],y=timestamp_all.rename('timestamp'), scale='area')
plt.xticks(range(2),labels=['Y','N'])
plt.show()

# EDA 추가

- 연소득
- 대출희망금액: 늘 b에 넣기
- 대출목적
- 기대출금액

In [ ]:
user_spec_train.corr()

In [ ]:
loan_result_train.head()

In [ ]:
user_spec_train.head()

In [9]:
df = pd.merge(user_spec_train,loan_result_train[['application_id','is_applied']] , how='inner')

In [ ]:
df.shape

In [13]:
df_drop = df.drop_duplicates()

In [ ]:
df_drop.shape

In [ ]:
df_drop.corr()

In [ ]:
cat = ['insert_time', 'income_type', 'employment_type','houseown_type','purpose','gender']
con = ['credit_score','yearly_income','desired_amount','personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn', 'existing_loan_cnt','existing_loan_amt']
y = 'is_applied'



import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(3,5,figsize=(15,8))
cond = df_drop[y]==1
for i, (var,ax) in enumerate(zip(cat+con,axes.ravel())):
    if i<=5:
        pass
    else:
        sns.boxplot(data = df_drop, x = 'is_applied', y = var, ax = ax)
plt.tight_layout()